# Stat 535 HW5
Dongyang Wang

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
%matplotlib inline

### Problem 1

In [ ]:
import requests

response = requests.get('https://sites.stat.washington.edu/mmp/courses/535/fall22/assignments/hw4-nn-train-100.dat')
hw4_nn = response.text

data_by_line = hw4_nn.split('\n')
data_by_line = data_by_line[:len(data_by_line)-1]

for i in range(0,len(data_by_line)):
    data_by_line[i] = ' '.join(data_by_line[i].split())
    data_by_line[i] = data_by_line[i].split(' ')

hw4_nn = [0] * len(data_by_line)
    
for i in range(len(data_by_line)):
    hw4_nn[i] = [float(j) for j in data_by_line[i]]

nn_train = pd.DataFrame(hw4_nn, columns =['x1', 'x2', 'y'])

In [ ]:
import requests

response = requests.get('https://sites.stat.washington.edu/mmp/courses/535/fall22/assignments/hw4-nn-test.dat')
hw4_nn = response.text

data_by_line = hw4_nn.split('\n')
data_by_line = data_by_line[:len(data_by_line)-1]

for i in range(0,len(data_by_line)):
    data_by_line[i] = ' '.join(data_by_line[i].split())
    data_by_line[i] = data_by_line[i].split(' ')

hw4_nn = [0] * len(data_by_line)
    
for i in range(len(data_by_line)):
    hw4_nn[i] = [float(j) for j in data_by_line[i]]

nn_test = pd.DataFrame(hw4_nn, columns =['x1', 'x2', 'y'])

In [ ]:
plt.scatter(nn_train['x1'], nn_train['x2'], c = nn_train['y'])

plt.title("Neural Network by Class (Yellow for Positive)")
plt.show()

### b

No, it is impossible to do that because the decision boundary is irregular and we need way more than 2 units to perfectly seperate the +1 and -1 cases. So we will need more than 2 units in the layer to get perfect classification where $\hat{L}_{01} = 0$.

### c

I have chosen the initial points from a uniform distribution with a small interval, namely [-0.5,0.5]. This way, I randomly generate my initial points and not to get too extreme values for the sigmoid functions.

Note that I have ignored the tolerance becuase the change in the beta and w values are very small.

In [ ]:
X = []
Y = []
#if nn_train['y'][i] == 1:
#        Y.append([nn_train['y'][i]])
#    else:
#        Y.append([0])
for i in range(len(nn_train)):
    Y.append([nn_train['y'][i]])
    X.append([1, nn_train['x1'][i], nn_train['x2'][i]])

Y = np.matrix(Y)
X = np.matrix(X)

X_test = []
Y_test = []
for i in range(len(nn_test)):
    Y_test.append([nn_test['y'][i]])
    X_test.append([1, nn_test['x1'][i], nn_test['x2'][i]])
Y_test = np.matrix(Y_test)
X_test = np.matrix(X_test)

In [ ]:
def get_gradient_beta(y, yhat):
    sub = np.subtract(y, yhat)
    return sub.mean()

def get_gradient_w(y, yhat, x):
    y_yhat = np.subtract(y, yhat)
    res = np.matmul(y_yhat, x)
    return res.mean()

def prediction(b, w, x, b0):
    xt = x.transpose()
    wxt = np.matmul(w, xt)
    z = 1/(1+np.exp(wxt))
    bt = b.transpose()
    val = np.matmul(bt, z)[0,0]+b0
    return  1/(1+np.exp(val))

#def get_logit(y, w, x):
#    exponent = -1 * y[0,0] * prediction(b, w, x, b0)
#    return np.log(1 + np.exp(exponent))

def get_logit_one_w(y, wi, x):
    xt = x.transpose()
    wxt = np.matmul(wi, xt)[0,0]
    exponent = -1*(y[0,0]*wxt)
    return np.log(1 + np.exp(exponent))

def get_logit(y, w, x):
    logit = 0
    for wi in w:
        logit += abs(get_logit_one_w(y, wi, x))
    return logit

### Problem 2

#### a

Based on definition, $J_{\lambda1} = \hat{L}(w_1) + \frac{\lambda_1}{2} ||w_1||^2 < \hat{L}(w_2) + \frac{\lambda_1}{2} ||w_2||^2$, similarly, $J_{\lambda2} = \hat{L}(w_2) + \frac{\lambda_2}{2} ||w_2||^2 < \hat{L}(w_1) + \frac{\lambda_2}{2} ||w_1||^2$. Since 
$\lambda_1 > \lambda_2$, $\hat{L}(w_2) + \frac{\lambda_2}{2} ||w_2||^2 < \hat{L}(w_1) + \frac{\lambda_2}{2} ||w_1||^2 < \hat{L}(w_1) + \frac{\lambda_1}{2} ||w_1||^2 < \hat{L}(w_2) + \frac{\lambda_1}{2} ||w_2||^2$. Assume, to the contrary, $||w_1|| > ||w_2||$, $\hat{L}(w_2) + \frac{\lambda_2}{2} ||w_2||^2 < \hat{L}(w_2) + \frac{\lambda_1}{2} ||w_2||^2 < \hat{L}(w_2) + \frac{\lambda_1}{2} ||w_1||^2$ and therefore $\frac{\lambda_2}{2} ||w_2||^2 > \frac{\lambda_1}{2} ||w_1||^2$. Thus, $\frac{||w_1||^2}{||w_2||^2} < \frac{\lambda_2}{\lambda_1}$, which is a contradiction. Therefore, $||w_1|| < ||w_2||$.

In ridge regression, for example,

$\hat{w} = (X^TX + \lambda I_n)^{-1}X^TY$. We can observe that with an increase in $\lambda$, The inverse of $X^TX + \lambda I_n$ decreases, therefore regularized quantity dedcreases as more regularization is imposed.

#### b

Since $\lambda_1 > \lambda_2$, and proved above, $||w_2|| > ||w_1||$. Given $\hat{L}(w_2) + \frac{\lambda_2}{2} ||w_2||^2 < \hat{L}(w_1) + \frac{\lambda_2}{2} ||w_1||^2$, $\hat{L}(w_2) - \hat{L}(w_1) < \frac{\lambda_2}{2} ||w_1||^2 - \frac{\lambda_2}{2} ||w_2||^2<0$ and therefore $\hat{L}(w_2) < \hat{L}(w_1)$.

In ridge regression, for example,

$\hat{L} = \sum (y - w^T x)^2$ and with an increase in regularization lambda there is decrease in regularized quantity w, such that $y - w^T x$ increases and so is $\hat{L}$.